In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

In [2]:
loader = CSVLoader(file_path='../data/papers_data.csv')
data = loader.load()

In [3]:
print(data)

[Document(metadata={'source': '../data/papers_data.csv', 'row': 0}, page_content='Supervisor\'s name: Jaroslaw Drapala\ninterests: mathematical modeling dynamical systems machine learning exploratory data analysis\nresearch papers: {"Mammalian Host-Versus-Phage immune response determines phage fate in vivo": "emerging bacterial antibiotic resistance draws attention bacteriophages therapeutic alternative treat bacterial infection examples phage combat bacteria abound however despite careful testing antibacterial activity vitro failures nevertheless commonly occur investigated immunological response phage antibacterial potency vivo anti-phage activity phagocytes antibodies serum complement identified direct testing high-resolution fluorescent microscopy accommodated experimental data mathematical model propose universal schema innate adaptive immunity impact phage pharmacokinetics based results numerical simulations found mammalian-host response infecting bacteria causes concomitant remo

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [5]:
retriever = db.as_retriever(search_kwargs={'k': 5})

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0.2),
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
)

In [7]:
prompt_template = """
You are an expert in providing information about thesis supervisors at Politechnika Wrocławska.
When given a user question about the most suitable thesis supervisor for their project based on their reasearch papers and intrests, provide the supervisor's name along with titles of their research papers.

User Question:
{question}

Response:
"""

In [8]:
question = "Who should I pick as promotor for my engineering thesis 'Train Rescheduling and Track Closure Optimization'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)['result'])


I am sorry, I cannot provide a helpful answer as none of the supervisors listed have research papers related to train rescheduling or track closure optimization.


In [9]:
question = "Who should I pick as promotor for my engineering thesis 'Heuristic Methods for Carpooling Optimization: Balancing Economic, Comfort, Safety, and Social Criteria'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)['result'])


Supervisor's name: Janusz Sobecki
Research papers:
- "Application of hybrid recommendation in web-based cooking assistant"
- "Implementations of web-based recommender systems using hybrid methods."
- "Student courses recommendation using ant colony optimization"
- "Using consensus methods to construct adaptive interfaces in multimodal web-based systems"
- "Using eyetracking in a mobile applications usability testing"


In [10]:
question = "Who should I pick as promotor for my engineering thesis 'The Role of Recomendation Systems in Ecommerce?'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)['result'])


Based on your research topic, the most suitable supervisor for your engineering thesis would be Janusz Sobecki. His research interests in human-computer interaction, eye tracking, and multimedia recommendation systems align well with your topic. Some of his relevant research papers include "Application of hybrid recommendation in web-based cooking assistant", "Implementations of web-based recommender systems using hybrid methods", and "Using eyetracking in a mobile applications usability testing".
